In [4]:
import numpy as np 
import matplotlib.pyplot as plt
from numpy import *
import random

from sklearn import datasets 
from sklearn.datasets import load_iris

from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from sklearn.tree import export_text
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor 

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor

from sklearn.decomposition import PCA


def read_data(run_num, prob):

    normalise = False
    
    if prob == 'classifification': 
        # Source: Pima-Indian diabetes dataset - https://www.kaggle.com/kumargh/pimaindiansdiabetescsv
        data_in = genfromtxt("datasets/pima.csv", delimiter=",")
        data_inputx = data_in[:, 0:8]  # all features 0-7 
        data_inputy = data_in[:, -1]   # this is target - so that last col is selected from data

    elif prob == 'regression':   # energy - regression prob
        data_in = genfromtxt('datasets/energy/ENB2012_data.csv', delimiter=",")  # you can replace this with Abalone
        data_inputx = data_in[:, 0:8]  # all features 0-7
        data_inputy = data_in[:, 8]    # this is target - just the heating load selected from data  

    if normalise == True:
        transformer = Normalizer().fit(data_inputx)    # fit does nothing
        data_inputx = transformer.transform(data_inputx)
 
    x_train, x_test, y_train, y_test = train_test_split(data_inputx, data_inputy, test_size=0.40, random_state=run_num)

    return x_train, x_test, y_train, y_test


def dimen_reduction(x_train, x_test, type_model):

    # Scikit-learn is using SVD for PCA

    if type_model == 0:   # SVD solver - full
        pca = PCA(n_components=3, svd_solver='full')    # SVD LAPACK Solver
        
    elif type_model == 1: # SVD 
        pca = PCA(n_components=3, svd_solver='arpack')  # SVD AROACK Solver

    # note number of components can be changed to 0.95
    # but since we have different train and test data, that can create problems
    # it is best to combine both train and test data and then split them back again

    #data = np.vstack((x_train, x_test))   # something along these lines

    #print(data.shape, ' * ')

    reduced_datatrain = pca.fit_transform(x_train)
    train_varianceratio = pca.explained_variance_ratio_

    reduced_datatest = pca.fit_transform(x_test)
    test_varianceratio = pca.explained_variance_ratio_

    return reduced_datatrain, reduced_datatest, test_varianceratio, train_varianceratio
 
    
def scipy_models(x_train, x_test, y_train, y_test, type_model, hidden, learn_rate, run_num, problem): 

    if problem == 'classifification':
        if type_model == 0:   # SGD 
            model = MLPClassifier(hidden_layer_sizes=(hidden,), random_state=run_num, 
                                  max_iter=100, solver='sgd', learning_rate_init=learn_rate )
            
        elif type_model == 1:  # https://scikit-learn.org/stable/modules/tree.html (see how tree can be visualised)
            model = DecisionTreeClassifier(random_state=0, max_depth=tree_depth) 
            
        elif type_model == 2:
            model = RandomForestClassifier(n_estimators=100, max_depth=tree_depth, random_state=run_num)             

    elif problem == 'regression':
        if type_model == 0:    # SGD  
            model = MLPRegressor(hidden_layer_sizes=(hidden*3,), random_state=run_num, 
                                 max_iter=500, solver='adam', learning_rate_init=learn_rate) 
            
        elif type_model == 1:  
            model = DecisionTreeRegressor(random_state=0, max_depth=tree_depth)
            
        elif type_model == 2: 
            model = RandomForestRegressor(n_estimators=100, max_depth=tree_depth, random_state=run_num)        
   

    # Train the model using the training sets
    model.fit(x_train, y_train)   

    if type_model == 1:
        r = export_text(model)
        print("\n", r)

    # Make predictions using the testing set
    y_pred_test = model.predict(x_test)
    y_pred_train = model.predict(x_train) 

    if problem == 'regression':
        perf_test = np.sqrt(mean_squared_error(y_test, y_pred_test)) 
        perf_train = np.sqrt(mean_squared_error(y_train, y_pred_train)) 

    if problem == 'classifification': 
        perf_test = accuracy_score(y_pred_test, y_test) 
        perf_train = accuracy_score(y_pred_train, y_train) 
        cm = confusion_matrix(y_pred_test, y_test) 
        #print(cm, 'is confusion matrix')
        #auc = roc_auc_score(y_pred, y_test, average=None) 

    return perf_test #,perf_train


def main(): 

    max_expruns = 5

    nn_all = np.zeros(max_expruns) 
    nnpca_all = np.zeros(max_expruns)   

    learn_rate = 0.01
    hidden = 8

    prob = 'classifification' # classification or regression 
    #prob = 'regression'       # classification or regression 

    # classifcation accurary is reported for classification and RMSE for regression

    print(prob, ' is our problem\n') 
 
    for run_num in range(0, max_expruns): 
        
        print(run_num, " experiment:\n")

        x_train, x_test, y_train, y_test = read_data(run_num, prob)   
        
        acc_nn = scipy_models(x_train, x_test, y_train, y_test, 0, hidden, learn_rate, run_num, prob) # SGD 

        # 0 is for PCA (you can try 1 for case of SVD)
        [reduced_datatrain, reduced_datatest, variance_scoretrain, variance_scoretest] = dimen_reduction(x_train, x_test, 1) 

        print(reduced_datatrain, 'reduced_datatrain\n')
        print(variance_scoretrain, 'variance_scoretrain\n')

        # SGD after PCA
        acc_nnpca = scipy_models(reduced_datatrain, reduced_datatest, y_train, y_test, 0, hidden, learn_rate, run_num, prob) 
               
        nn_all[run_num] = acc_nn
        nnpca_all[run_num] = acc_nnpca
        
    print(nn_all,' nn_all\n')
    print(np.mean(nn_all), ' mean nn_all\n')
    print(np.std(nn_all), ' std nn_all\n')
 
    print(nnpca_all,  ' tree_all\n')
    print(np.mean(nnpca_all),  ' mean - nn pca _all\n')
    print(np.std(nnpca_all),  ' std - nn - pca _all\n') 
    
    
if __name__ == '__main__':
     main() 

classifification  is our problem

0  experiment:

[[-10.92272505  -9.29478857  -6.21110455]
 [-35.28477004  38.86293517  -4.954755  ]
 [ 51.1500123  -14.677888    13.19876093]
 ...
 [-93.06118111  22.15037032   2.46565424]
 [-95.11956802  25.60355968  -3.82380589]
 [-90.63083864 -28.27771986  -4.57548287]] reduced_datatrain

[0.8572631  0.07561149 0.03582277] variance_scoretrain

1  experiment:

[[129.46964732  11.63017101   2.11365406]
 [208.57798741  28.2577262  -10.9060684 ]
 [-77.08269566  -2.26574547 -23.36877114]
 ...
 [-75.24592199 -17.11965938 -18.38085852]
 [-70.71684327 -56.47851787   5.98185911]
 [-75.62232152  10.45904004 -15.93669904]] reduced_datatrain

[0.8976209  0.0577072  0.02344864] variance_scoretrain

2  experiment:

[[-90.96844516  20.53603686  -8.49300816]
 [102.15605047 -10.80632781   7.63984192]
 [-91.62187553   6.0037081   -4.08060999]
 ...
 [ 32.23782907  -0.93314994   0.63300589]
 [ 14.35958941   8.59213335  -1.48182944]
 [-91.47179301   3.11921876   6.11913